After take a look into the data distribution of DonorsChoose.org, I found that I couldn't build a simple model that applied features such as grades, subject categories, states, and number of previously posted projects, to make precise prediction of whether the project would get approved or not. So now I'm going to dig into how the project title and project essays make people decide to support it.

### Load data and import modules

In [ ]:
#load modules
import nltk #for tokenizing
import tqdm
#https://github.com/tqdm/tqdm 第三方進度條模組
import numpy as np
import pandas as pd
#PunktSentenceTokenizer
nltk.download("punkt")
import zipfile
import gc

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
print(os.listdir("../input/glove6b300dtxt"))
os.listdir("../input/donorschoose-application-screening")

In [ ]:
#load data
df_train = pd.read_csv('../input/donorschoose-application-screening/train.csv')
df_test = pd.read_csv('../input/donorschoose-application-screening/test.csv')
df_resources = pd.read_csv('../input/donorschoose-application-screening/resources.csv')

### Data types and columns

**Text**: project_title, project_essay_1, project_essay_2, project_essay_3, project_essay_4, project_resource_summary in **train/test**, and description in **resources**

**Class/Label**: teacher_id, teacher_prefix, project_grade_category, project_subject_categories, project_subject_subcategories in **train/test**

**Quantity**: quantity and price in **resources**, and teacher_number_of_previously_posted_projects in **train/test**

## Deal with Text data

#### 1. Load the pre-trained embedded word vectors

In [ ]:
#load word embedded vectors
import os
embedding_path = "../input/glove6b300dtxt/glove.6B.300d.txt"

def read_embedding_vec(embedding_file_path):
    embedding_list = []
    embedding_word_dict = dict()
    
    f = open(embedding_file_path)

    for index, line in enumerate(f):
        #the line contains target word and its word vector, so split it first
        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except:
            continue
        #save the word vec into a list
        embedding_list.append(coefs)
        #build a connection between word and its word id (0, 1, 2, ......)
        embedding_word_dict[word] = len(embedding_word_dict)
    f.close()
    embedding_list = np.array(embedding_list)
    return embedding_list, embedding_word_dict

In [ ]:
#load embedding word vectors
embedding_list, embedding_word_dict = read_embedding_vec(embedding_path)
#embedding_list[:5], embedding_word_dict['the']

In [ ]:
#set word_vector for UNKNOWN_WORD and _NAN_
try:
    embedding_word_dict['nanword']
except:
    embedding_word_dict['nanword'] = len(embedding_word_dict)
    embedding_list = np.append(embedding_list, [np.zeros_like(embedding_list[0,:])], axis = 0)
try:
    embedding_word_dict['unknownword']
except:
    embedding_word_dict['unknownword'] = len(embedding_word_dict)
    embedding_list = np.append(embedding_list, [np.zeros_like(embedding_list[0,:])], axis = 0)
#(embedding_word_dict['nanword'], embedding_word_dict['unknownword'], len(embedding_word_dict)), embedding_list[-3:]

Now we have loaded the pre-trained word vectors, the next step is to transform our project titles and essays. Note that the word embedding does not include "can't", "couldn't", "wouldn't", and things like those, we need to separate the not things when doing tokenization.

The sentences include some informal words and some unclear/meaningless punctuation. They should be replaced with sth making more sense to the sentences.

#### 2. Define preprocessing procedures

In [ ]:
import re
def preprocess(string):
    '''
    :param string:
    :return:
    '''
    string = string.lower()
    string = re.sub(r'(\\")', '', string)
    string = re.sub(r'(\\r\\n)', ' ', string)
    string = re.sub(r'(\\r)', ' ', string)
    string = re.sub(r'(\\n)', ' ', string)
    string = re.sub(r'(\\)', ' ', string)
    string = re.sub(r'\\t', ' ', string)
    string = re.sub(r'\:', ' ', string)
    string = re.sub(r'\"\"\"\"', ' ', string)
    string = re.sub(r'_', ' ', string)
    string = re.sub(r'\+', ' ', string)
    string = re.sub(r'\=', ' ', string)
    string = re.sub(r'\-', ' ', string)
    #sep numstring to "num string"
    string = " ".join(re.split(r'(\d+)', string))
    string = string.strip()
    return string

#### 3. Define tokenization procedures

In [ ]:
words_dict = dict()
#for sentences checking
#for i in range(20,50):
#    print(df_train['project_title'][i])
#tokenize_sentences(df_train['project_title'], words_dict, early_stopping = False)

Before starting tokenizing, I find out that there are data differences in the datasets: Starting on May 17, 2016, the number of essays was reduced from 4 to 2, and the questions of the essays are different as well.

    project_essay_1: "Introduce us to your classroom"
    project_essay_2: "Tell us more about your students"
    project_essay_3: "Describe how your students will use the materials you're requesting"
    project_essay_4: "Close by sharing why your project will make a difference"
    
    Starting on May 17, 2016, the number of essays was reduced from 4 to 2, and the prompts for the first 2 essays were changed to the following:

    project_essay_1: "Describe your students: What makes your students special? Specific details about their background, your neighborhood, and your school are all helpful."
    project_essay_2: "About your project: How will these materials make a difference in your students' learning and improve their school lives?"
    
I think the essay 1 and 2 before May 17, 2016, can be combined into the idea of essay 1 after May 17, 2016. Same idea for essay 3 and 4. So the sentences should get combination before tokenization. Also, to build a simpler model, I will try to combine all text data into one column and do it with LSTM or GRU model.

In [ ]:
#combine the sentences
def combine_text(df):
    try:
        if len(df['text']) == len(df): return df
    except: pass
    df['text'] = ""
    text_col  = ['project_title', 'project_essay_1', 
                 'project_essay_2', 'project_essay_3', 
                 'project_essay_4', 'project_resource_summary']
        
    #find nan values and apply fillna
    df[text_col] = df[text_col].fillna('NANWORD')
    df['text'] = df.apply(lambda x: " ".join([str(x[col]).strip() for col in text_col]), axis=1)
    #df.drop(['project_title', 'project_essay_1', 
    #         'project_essay_2', 'project_essay_3', 
    #         'project_essay_4', 'project_resource_summary'], axis = 1, inplace = True)
    return df

In [ ]:
#df_train.drop('text', axis = 1, inplace = True)
df_train = combine_text(df_train)
df_train.head()

In [ ]:
#print(df_train.iloc[0, -3])
#print(df_train.iloc[0, -2])

In [ ]:
def tokenize_sentences(sentences, words_dict, early_stopping = False):
    '''
    read sentences from the dataset and return tokenized_sentences and local words_dict
    early_stopping is set to run small dataset to check how the tokenization goes
    '''
    tokenized_sentences = []
    for sentence in tqdm.tqdm(sentences):
        #check attribute first
        if hasattr(sentence, "decode"): 
            sentence = sentence.decode("utf-8")
        #run preprocessing
        sentence = preprocess(sentence)
        #start tokenizing
        tokens = nltk.tokenize.word_tokenize(sentence)
        result = []
        for word in tokens:
            word = word.lower()
            if word not in words_dict:
                words_dict[word] = len(words_dict)
            word_index = words_dict[word]
            result.append(word_index)
        tokenized_sentences.append(result)
        if early_stopping:
            print(sentence)
            if len(tokenized_sentences) == 50:
                return tokenized_sentences, words_dict
    return tokenized_sentences, words_dict

In [ ]:
#start tokenization 
df_train['tokenized_text'], words_dict = tokenize_sentences(df_train['text'], words_dict)

In [ ]:
#count the sentence length for each project
df_train["tokenized_text_length"] = df_train['tokenized_text'].apply(lambda x: len(x))

### Now we finish the tokenization on training set!
Since I didn't see any huge differences in individual project title and essay, and all the texts should be viewed as a whole context item, I'm going to check whether length of the context have impacts on the approval. Also, I would like to check the length distribution and decide whether I should set the length maximum for the text data. If the differences of the text length are small, maybe I can apply CNN model and the idea of BoW(Bag-of-Words) to make the training phrase much faster.

In [ ]:
plt.hist(df_train[df_train["project_is_approved"] == True]["tokenized_text_length"], 
         histtype = 'bar', bins=50, range = (150, 650),
         label = "Approved",
         color = 'red', stacked = True)
plt.hist(df_train[df_train["project_is_approved"] == False]["tokenized_text_length"], 
         histtype = 'bar', bins=50, range = (150, 650),
         label = "Not approved",
         color = 'green', stacked = True)
plt.legend()

In [ ]:
len(df_train[(df_train["project_is_approved"] == True) & (df_train["tokenized_text_length"]>= 630)])

Well, the length doesn't mean much to the approvals, since both approved and unapproved have the same data distribution.

#### 4. Lookup the words_dict onto embedded_words_dict
Although we have a huge pre-trained embedded word vectors, but we might not need them all, so now I would like to check how many words will be used in the dataset.

In [ ]:
def clear_embedding_list(embedding_list, embedding_word_dict, words_dict):
    '''
    The dataset might not use all the pre-trained word vectors, we only need the words included in the dataset.
    '''
    cleared_embedding_list = []
    cleared_embedding_word_dict = {}

    for word in words_dict:
        if word not in embedding_word_dict:
            continue
        word_id = embedding_word_dict[word]
        row = embedding_list[word_id]
        cleared_embedding_list.append(row)
        cleared_embedding_word_dict[word] = len(cleared_embedding_word_dict)

    return cleared_embedding_list, cleared_embedding_word_dict

In [ ]:
words_dict['nanword']

In [ ]:
def convert_tokens_to_embedded(tokenized_sentences, words_to_ids, embedding_word_dict, 
                          sentences_length = 630, padding = embedding_word_dict['nanword'], 
                               early_stopping = False):
    '''
    tokenized_sentences and embedding_word_dict have different word ids, this function would try to look them up
    Input: local tokenized sentence
    Output: Lookup embedded tokenized sentence
    '''
    words_train = []
    #loop all the sentences of the local dataset
    for sentence in tokenized_sentences:
        current_words = []
        for word_index in sentence:
            word = words_to_ids[word_index]
            #why -1? -1 to get index and len(embedding_word_dict) = 400001, 
            #cleared_embedding_word_dict['unknownword'], cleared_embedding_word_dict['nanword']
            #look up to embedding_word_dict to find the word index
            word_id = embedding_word_dict.get(word, embedding_word_dict['unknownword'])
            current_words.append(word_id)
        #set maximum for sentence length
        if len(current_words) >= sentences_length:
            current_words = current_words[:sentences_length]
        else:
            #tokenized_sentences length is less than the setup sentence length => add NANWORD
            current_words += [padding] * (sentences_length - len(current_words))
        if early_stopping and len(words_train) >= 10:
            break
        words_train.append(current_words)
    return words_train

In [ ]:
words_to_ids = dict()
for key, value in words_dict.items():
    words_to_ids[value] = key

In [ ]:
train_x_words = np.array(convert_tokens_to_embedded(df_train['tokenized_text'], 
                words_to_ids, embedding_word_dict, 
                sentences_length = 630))

In [ ]:
#print(' '.join(words_to_ids[word] for word in df_train['tokenized_text'][0]))

Note that lists in DataFrame are hard to convert the whole column without looping(I couldn't find any ways to fix that).

In [ ]:
#df_train.drop('lookup_tokenzied_sentence', axis = 1, inplace = True)
df_train['tokenized_text'].head()

In [ ]:
#check code
#" ".join([embedding_ids_dict[word_idx] for word_idx in convert_tokens_to_ids(df_train['tokenized_text'], 
#                                      words_to_ids, embedding_word_dict, 
#                                      early_stopping = True)[0]])
#Looks pretty good!

Now we have got the embedded word vector lookup done, let's try to train the model next.

In [ ]:
#df_train.head()

In [ ]:
#drop some unused columns to release memory
df_train.drop(['project_title', 'project_essay_1', 'project_essay_2', 'project_essay_3', 'project_essay_4',
              'project_resource_summary', 'text'], axis = 1, inplace = True)
gc.collect()

## Update: Add numerical data into the model and train an mixed model
From the baseline model, we also see that the total price of the project and the number of previously posted projects can easily make a nice prediction. So I would like to add the numerical nodes into the last dense DNN model and train the model again to see if it can do better prediction than the first try (0.77703). Note to shrink the model computation and make the numerical data have more power on it, I adjust the dense layers from [64, 32] nodes down to [64, 16].

In [ ]:
#note that price is only for one quantity of the resource
df_resources['total_price'] = df_resources['quantity'] * df_resources['price']
sum_total_price = pd.DataFrame(df_resources.groupby('id').total_price.sum()).reset_index()
#sum_total_price.head()

In [ ]:
#append resources items to train and test
df_train = pd.merge(df_train, sum_total_price, on='id')
df_test = pd.merge(df_test, sum_total_price, on = 'id')

In [ ]:
train_x_num = np.array(df_train[['total_price', 'teacher_number_of_previously_posted_projects', 'tokenized_text_length']])
#import scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

train_x_num = scaler.fit_transform(train_x_num)

### Train a GRU model

In [ ]:
#import modules from keras
from keras.layers import Input, Dense, Embedding, Flatten, concatenate, Dropout, Convolution1D, \
                         GlobalMaxPool1D, GlobalAveragePooling1D, SpatialDropout1D, CuDNNGRU, Dropout,\
                         Bidirectional, GRU, BatchNormalization, AveragePooling1D, MaxPooling1D
from keras.models import Model, Sequential
from keras import optimizers

In [ ]:
def get_model(embedding_matrix = embedding_list, num_features = 3,
              maxlen = 630, dropout_rate = 0.4, learning_rate = 5e-3,
              recurrent_units = 64):
    '''
    embedding_matrix needs to be an array
    '''
    input_words = Input((maxlen, ))
    
    input_nums = Input((num_features, ))
    #set dim for embedded word
    embedding_dim = 300
    max_features = len(embedding_matrix)
    
    x_words = Embedding(max_features, embedding_dim,
                        weights = [embedding_matrix], 
                        input_length = maxlen,
                        trainable=False)(input_words)
    x_words = SpatialDropout1D(dropout_rate)(x_words)
    
    x_words1 = Bidirectional(GRU(recurrent_units, return_sequences=True))(x_words)
    #set filters to be 128, kernel_size to be 5
    x_words1 = Convolution1D(64, 5, activation="relu")(x_words1)
    x_words1_1 = MaxPooling1D(pool_size = 2, strides = 2, padding = 'same')(x_words1)
    x_words1_1 = GlobalAveragePooling1D()(x_words1)
    
    #Do we really need the CNN model to make the prediction better?
    #x_words2 = Convolution1D(32, 5, activation="relu")(x_words)
    #x_words2 = MaxPooling1D(2,padding='same')(x_words2)
    #x_words2 = Convolution1D(64, 5, activation="relu")(x_words2)
    #x_words2_1 = MaxPooling1D(2,padding='same')(x_words2)
    #x_words2_1 = GlobalAveragePooling1D()(x_words2_1)
    
    #x = concatenate([x_words1_1, x_words2_1])
    #print(x.shape)
    
    #try if only GRU can output a great score
    x = BatchNormalization()(x_words1_1)
    
    dense_units = [64, 16] #Ver8 & Ver9
    #dense_units = [32, 8] #Ver11
    #x = Flatten()(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(dense_units[0], activation="relu")(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(dense_units[1], activation="relu")(x)
    
    #join x_nums
    x_nums = input_nums
    #since x might have some gradient vanishing situations already, I would like to do batchnorm again before concatenate with x_nums
    x = BatchNormalization()(x)
    x = concatenate([x, x_nums])
    predictions = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=[input_words, input_nums], outputs=predictions)
    model.compile(optimizer=optimizers.Adam(learning_rate, decay=1e-6),
              loss='binary_crossentropy',
              metrics=['accuracy'])

    return model

In [ ]:
#del model
#del get_model
#gc.collect()
model = get_model()

In [ ]:
from keras.callbacks import *
from sklearn.metrics import roc_auc_score
file_path = 'best.h5'

In [ ]:
#test = np.array(train_x_words)
#test.shape
#train_x_words.shape

In [ ]:
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True,
                             save_weights_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=4)
lr_reduced = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1,
                               epsilon=1e-4, mode='min')

callbacks_list = [checkpoint, early, lr_reduced]

history = model.fit([train_x_words, train_x_num], 
                    df_train['project_is_approved'], validation_split=0.03,
                    verbose=1, callbacks=callbacks_list, epochs=10, batch_size=256)

### Test data preprocessing

After done model training, now I'm going to to text lookup for the test dataset. Here are the procedures:
    
1. Do sentence combination and tokenization on test dataset
2. Look up the tokenized words onto the words_dict
3. Run prediction with the processed test data

Since I train the NLP model with words appeared in the training dataset, the first idea come to me is that the test dataset may only check whether the word shows up in the training words_dict. But the word vectors are imported from pre-trained model, maybe it's better to refer directly to the embedded_words_dict when doing prediction.

In [ ]:
#del data only for training to release memory
del df_train
#del cleared_embedding_word_dict
#del cleared_embedding_list
del train_x_words
gc.collect()

In [ ]:
# test data preprocessing
df_test = combine_text(df_test)
# drop other text columns directly
df_test.drop(['project_title', 'project_essay_1', 'project_essay_2', 'project_essay_3', 'project_essay_4',
             'project_resource_summary'], axis = 1, inplace = True)
gc.collect()

In [ ]:
# test text tokenization
df_test['tokenized_text'], words_dict = tokenize_sentences(df_test['text'], words_dict)
df_test["tokenized_text_length"] = df_test['tokenized_text'].apply(lambda x: len(x))

In [ ]:
# test_x_num
test_x_num = np.array(df_test[['total_price', 'teacher_number_of_previously_posted_projects', 'tokenized_text_length']])
test_x_num = scaler.transform(test_x_num)

In [ ]:
#words_to_ids = dict()
#renew words_to_ids
for key, value in words_dict.items():
    words_to_ids[value] = key

In [ ]:
#test tokenized words vector lookup
test_x_words = np.array(convert_tokens_to_embedded(df_test['tokenized_text'], 
                words_to_ids, embedding_word_dict, 
                padding = embedding_word_dict['nanword'],
                sentences_length = 630))

In [ ]:
#run prediction
model.load_weights(file_path)
pred_test = model.predict([test_x_words, test_x_num], batch_size=1024, verbose=1)

df_test["project_is_approved"] = pred_test
#submit our prediction!
df_test[['id', 'project_is_approved']].to_csv("submission.csv", index=False)

#### Future goals:
1. Add numerical data into the model and train an mixed model
2. Experiment some hyperparameter tuning
3. Read through the 1st model and borrow some feature-engineering ideas into mine.  https://www.kaggle.com/shadowwarrior/1st-place-solution/notebook

## Update: Experiment some hyperparameter tuning
The model with just GRU and numerical items performed even better than the ensemble model with one more set of CNN layers. I think the main reason is that the length of the text varies a lot, and that makes the model put too much efforts on the padding words but learning nothing. So the CNN make the model kind of overfitting.
Newest try without CNN ensemble (ver.9): **0.79172**, slightly better and faster than the ensemble model (ver.8): **0.78922**
Also, I was thinking that if the model was overfitting and try to shrink the model, but that also reduced the score to **0.78847**.

Reference:

https://www.kaggle.com/fizzbuzz/beginner-s-guide-to-capsule-networks

https://www.kaggle.com/fizzbuzz/the-all-in-one-model